In [1]:

from openai import OpenAI
import asyncio
import configparser
# OpenAI API 설정
config = configparser.ConfigParser()
# DeepL API URL 및 인증 키
config.read('config.ini')
api_key = config['DEFAULT']['OPENAI_API_KEY']
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=api_key,
)



In [2]:
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [18]:
file = client.files.create(
  file=open("train.py", "rb"),
  purpose='assistants'
)

In [19]:

assistant = client.beta.assistants.create(
    name="Code Commenting Helper",
    instructions="You are a code commenting helper. When you receive a code file, add comments to it and return the file with the comments added.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
    file_ids=[file.id],
)


In [20]:
print(assistant)

Assistant(id='asst_fi8CRVRFabAWFMOkS4uQoHrY', created_at=1699794695, description=None, file_ids=['file-rPG4QkdmnEpVWE0lypuzANk3'], instructions='You are a code commenting helper. When you receive a code file, add comments to it and return the file with the comments added.', metadata={}, model='gpt-4-1106-preview', name='Code Commenting Helper', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])


In [21]:
file 

FileObject(id='file-rPG4QkdmnEpVWE0lypuzANk3', bytes=7187, created_at=1699794683, filename='train.py', object='file', purpose='assistants', status='processed', status_details=None)

In [22]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Please analyze the code, add comments in Korean, and return the file to me.",
      "file_ids": [file.id]
    }
  ]
)






In [23]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please analyze the code, add comments in Korean, and return the file to me.",
    file_ids=[file.id],
)

In [24]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  tools=[{"type": "code_interpreter"}, {"type": "retrieval"}]
)

In [25]:
run

Run(id='run_f4aLjHjYZk7BXRw71xrEGypA', assistant_id='asst_fi8CRVRFabAWFMOkS4uQoHrY', cancelled_at=None, completed_at=None, created_at=1699794841, expires_at=1699795441, failed_at=None, file_ids=['file-rPG4QkdmnEpVWE0lypuzANk3'], instructions='You are a code commenting helper. When you receive a code file, add comments to it and return the file with the comments added.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_jDmBdrhuL6gK1roa3JoY4rjp', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval')])

In [26]:
# Retrieve the message object
message = client.beta.threads.messages.retrieve(
  thread_id=thread.id,
  message_id=message.id
)

# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []



In [27]:
# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

    # Gather citations based on annotation attributes
    if (file_citation := getattr(annotation, 'file_citation', None)):
        print(file_citation)
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
    elif (file_path := getattr(annotation, 'file_path', None)):
        print(file_path)
        cited_file = client.files.retrieve(file_path.file_id)
        citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
        # Note: File download functionality not implemented above for brevity

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)


messages = client.beta.threads.messages.list(
  thread_id=thread.id
)


In [28]:
print(messages)


SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_hLlQWuSFfm7yIsckoeuLRcrN', assistant_id='asst_fi8CRVRFabAWFMOkS4uQoHrY', content=[MessageContentText(text=Text(annotations=[], value='It appears that there is an issue preventing me from accessing the file directly. However, you mentioned that it contains code which you would like to have analyzed and commented in Korean. If you could please re-upload the file or provide its content by copying and pasting it here, I will then proceed to analyze the code and add the requested comments.'), type='text')], created_at=1699794849, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_f4aLjHjYZk7BXRw71xrEGypA', thread_id='thread_jDmBdrhuL6gK1roa3JoY4rjp'), ThreadMessage(id='msg_UanIhzjzCZEtkjZAXIB052KN', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Please analyze the code, add comments in Korean, and return the file to me.'), type='text')], created_at=1699794833, file_ids=['

In [29]:
len(citations)

0